# NIMS-KISTI-NVIDIA Hackathon 

# Problem Statement 



## Agenda 
 - Introduction to FourCastNet
 - Configure FourCastNet
 - Typoon Dataset (JMA best track) 
 - ECMWF ERA5 dataset(CDS API)
 - custom interval 
 - inference 
 - post processing 
 




# Typoon Dataset

### get typoon data with JMA Besttrack(~2022)

In [ ]:
!wget -N -q -O bst_all.zip https://www.jma.go.jp/jma/jma-eng/jma-center/rsmc-hp-pub-eg/Besttracks/bst_all.zip

In [ ]:
!unzip bst_all.zip 

In [ ]:
!head bst_all.txt
!tail bst_all.txt

In [ ]:
!wget -O format.html https://www.jma.go.jp/jma/jma-eng/jma-center/rsmc-hp-pub-eg/Besttracks/e_format_bst.html

## Understand JMA best track dataset

In [ ]:

import pandas as pd
import numpy as np 

class JMA_BestTrack:
    def __init__(self, filename):
        self.typhoons = self.read_jma_best_track_data(filename)

    @staticmethod
    def read_jma_best_track_data(filename):
        with open(filename, 'r') as file:
            lines = file.readlines()

        typhoons = {}
        typhoon_id = None
        typhoon_data = []
        for line in lines:
            if line.startswith('66666'):
                if typhoon_id:
                    typhoons[typhoon_id] = typhoon_data

                typhoon_id = line[ 6:10] if line[ 6:10].strip() else None 
                num_data_lines = int(line[13:15]) if line[13:15].strip() else None 

                typhoon_data = {
                    'header'       : line.strip(),
                    'typhoon_id'   : line[ 6:10] if line[ 6:10].strip() else None ,
                    'num_lines'    : int(line[13:15]) if line[13:15].strip() else None ,
                    'cyclone_id'   : line[17:20] if line[17:20].strip() else None ,
                    'id_check'     : line[21:25] if line[21:25].strip() else None ,
                    'name'         : line[30:50].replace(' ','') if line[30:50].strip() else None ,
                    'data_lines': []
                }
            else:
                typhoon_data['data_lines'].append(line.strip())

        if typhoon_id:
            typhoons[typhoon_id] = typhoon_data

        return typhoons

    def parse_data_lines(data_lines):
        typhoon_data = []
        for line in data_lines:
            data = {
                'date'         : line[0:8],
                'grade'        : int(line[13:14]),
                'latitude'     : int(line[15:19])/10 if line[15:19].strip() else None ,
                'longitude'    : int(line[19:23])/10 if line[19:23].strip() else None , 
                'pressure'     : int(line[24:28]) if line[24:28].strip() else None ,
                'wind_speed'   : int(line[33:36]) if line[33:36].strip() else None ,
                '50kt_d'       : int(line[41:42]) if line[41:42].strip() else None ,            
                '50kt_l_radius': int(line[42:46]) if line[42:46].strip() else None ,
                '50kt_s_radius': int(line[47:51]) if line[47:51].strip() else None ,
                '30kt_d'       : int(line[52:53]) if line[52:53].strip() else None ,
                '30kt_l_radius': int(line[53:57]) if line[53:57].strip() else None ,
                '30kt_s_radius': int(line[58:62]) if line[58:62].strip() else None ,
            }
            typhoon_data.append(data)
        return typhoon_data


    def get_data(self, typhoon_id):
        if typhoon_id in self.typhoons:
            data_lines = self.typhoons[typhoon_id]['data_lines']
            return JMA_BestTrack.parse_data_lines(data_lines)  # Remove 'self' here
        else:
            return None

def get_trajectory(jma_besttrack, typhoon_id):
    return   jma_besttrack.get_data(typhoon_id)        
        
def print_typhoon_info(jma_besttrack, typhoon_id):
    typhoon_data = jma_besttrack.get_data(typhoon_id)

    if typhoon_data:
        print(f"Typhoon ID: {typhoon_id}")
        print(f"Name: {jma_besttrack.typhoons[typhoon_id]['name']}")
        #print(f"International ID: {jma_besttrack.typhoons[typhoon_id]['typhoon_id']}")
        #print(f"Cyclone Number ID: {jma_besttrack.typhoons[typhoon_id]['cyclone_id']}")
        #print(f"Cyclone Number ID: {jma_besttrack.typhoons[typhoon_id]['id_check']}")        
        print(f"Total Data Lines: {jma_besttrack.typhoons[typhoon_id]['num_lines']}")
        print("\nTyphoon Data:")
        print(typhoon_data)
        for data in typhoon_data:
            print(data)
 
    else:
        print(f"No data found for typhoon ID {typhoon_id}")



def get_all_data(jma_besttrack):
    all_data = []
    for typhoon_id in jma_besttrack.typhoons:
        typhoon_data = jma_besttrack.get_data(typhoon_id)
        if typhoon_data:
            all_data.extend(typhoon_data)
    return all_data       

import datetime

def get_all_typhoon_info(jma_besttrack, start_year=1980):
    typhoon_info_list = []
    for typhoon_id in jma_besttrack.typhoons:
        typhoon_data = jma_besttrack.get_data(typhoon_id)
        if typhoon_data:
            first_data = typhoon_data[0]  # Get the first data point for the typhoon
            last_data = typhoon_data[-1]  # Get the last data point for the typhoon
            valid_wind_speed_data = [data for data in typhoon_data if data['wind_speed'] is not None]

            if valid_wind_speed_data:
                highest_wind_speed_data = max(valid_wind_speed_data, key=lambda x: x['wind_speed'])
                highest_wind_speed = highest_wind_speed_data['wind_speed']
                grade_at_highest_wind_speed = highest_wind_speed_data['grade']
            else:
                highest_wind_speed = None
                grade_at_highest_wind_speed = None

            input_start_date = first_data['date']
            year = int(input_start_date[:2])
            if year >= 51:  # Assuming 1951 to 2023
                year += 1900
            else:
                year += 2000

            output_start_date = f"{year:04d}{input_start_date[2:]}"  # Format as yyyymmddhh

            if year >= start_year:
                input_last_date = last_data['date']
                year = int(input_last_date[:2])
                if year >= 51:  # Assuming 1951 to 2023
                    year += 1900
                else:
                    year += 2000

                output_last_date = f"{year:04d}{input_last_date[2:]}"  # Format as yyyymmddhh

                typhoon_info = {
                    'id': typhoon_id,
                    'dur': jma_besttrack.typhoons[typhoon_id]['num_lines'],
                    'start': output_start_date,
                    'end': output_last_date,
                    'latitude': first_data['latitude'],
                    'longitude': first_data['longitude'],
                    'lp': min(typhoon_data, key=lambda x: x['pressure'])['pressure'],
                    'hws': highest_wind_speed,
                    'grade': grade_at_highest_wind_speed
                }
                typhoon_info_list.append(typhoon_info)

    return typhoon_info_list

In [ ]:
# Example usage
filename = 'bst_all.txt'
jma_besttrack = JMA_BestTrack(filename) 

In [ ]:
typhoon_info_list=get_all_typhoon_info(jma_besttrack, start_year=2020)
print (len(typhoon_info_list))

In [ ]:

# Now you have a list of typhoon information
for typhoon_info in typhoon_info_list:
    print(typhoon_info)

In [ ]:
typhoon_info_list=get_all_typhoon_info(jma_besttrack, start_year=2018)
print (len(typhoon_info_list))

In [ ]:

# Now you have a list of typhoon information
for typhoon_info in typhoon_info_list:
    print(typhoon_info)

### Visualize Typoon

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def plot_trajectory(trajectory, ax, color):
    colors = ['green', 'orange', 'yellow', 'blue', 'red'] # Colors for grades 2, 3, 4, 5, 6
    grades = [2, 3, 4, 5, 6]
    grade_labels = ['TD', 'TS', 'STS', 'TY', 'L']

    for i in range(len(trajectory) - 1):
        start_point = trajectory[i]
        end_point = trajectory[i + 1]
        grade = start_point['grade']
        color = colors[grade - 2]
        linewidth = grade #/ 2

        lats = [start_point['latitude'], end_point['latitude']]
        lons = [start_point['longitude'], end_point['longitude']]
        ax.plot(lons, lats, marker='', linestyle='-', linewidth=linewidth, color=color, transform=ccrs.PlateCarree())

    # Plot start and end markers
    start_lat, start_lon = trajectory[0]['latitude'], trajectory[0]['longitude']
    end_lat, end_lon = trajectory[-1]['latitude'], trajectory[-1]['longitude']
    ax.plot(start_lon, start_lat, marker='^', markersize=3, color='black', transform=ccrs.PlateCarree())
    ax.plot(end_lon, end_lat, marker='*', markersize=3, color='black', transform=ccrs.PlateCarree())

    # Create legend
    legend_elements = [Line2D([0], [0], color=color, lw=2, label=f'{grade} - {label}') for grade, label, color in zip(grades, grade_labels, colors)]
    ax.legend(handles=legend_elements, loc='upper left')

def visualize_trajectories(trajectories, typhoon_ids, region='globe2d'):
    # Set up the map projection and extent
    if region == 'globe2d':
        projection = ccrs.PlateCarree(central_longitude=130)
        extent = [-180, 180, -90, 90]
        trunc_extent = None
    elif region == 'southeastasia':
        projection = ccrs.PlateCarree(central_longitude=130)
        extent = [90, 180, 0, 45]
        trunc_extent = [100, 180, 0, 50]
    elif region == 'globe3d':
        projection = ccrs.Orthographic(central_longitude=130, central_latitude=0)
        extent = [-180, 180, -90, 90]
        trunc_extent = None  # No truncation in globe3d

    fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': projection})
    if trunc_extent:
        ax.set_extent(trunc_extent, crs=ccrs.PlateCarree())
    else:
        ax.set_global()  # Show the full globe
    ax.add_feature(cfeature.COASTLINE, edgecolor='yellow')
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.gridlines(color='black', linestyle='dashed')

    colors = ['green', 'orange', 'yellow', 'blue', 'red']  # Colors for grades 2, 3, 4, 5, 6

    for typhoon_id, trajectory in zip(typhoon_ids, trajectories):
        grade = trajectory[0]['grade']
        color = colors[grade - 2]
        plot_trajectory(trajectory, ax, color)

    plt.show()

In [ ]:
typhoon_ids = ['9208', '9209', '9915']
trajectories = [get_trajectory(jma_besttrack, typhoon_id) for typhoon_id in typhoon_ids]
visualize_trajectories(trajectories, typhoon_ids)

In [ ]:
typhoon_ids  = [  '9208', '9209','9915' ]
trajectories = [get_trajectory(jma_besttrack, typhoon_id) for typhoon_id in typhoon_ids]
visualize_trajectories(trajectories, typhoon_ids, region='southeastasia')

In [ ]:
typhoon_ids  = [  '9208', '9209','9915' ]
trajectories = [get_trajectory(jma_besttrack, typhoon_id) for typhoon_id in typhoon_ids]
visualize_trajectories(trajectories, typhoon_ids, region='globe3d')

## Tyophoon Hinamno(힌남노)

In [ ]:
typhoon_ids  = [ '2211' ]
trajectories = [get_trajectory(jma_besttrack, typhoon_id) for typhoon_id in typhoon_ids]
visualize_trajectories(trajectories, typhoon_ids, region='southeastasia')

## Cluster Analysis for (optional)
 -  start point, level and month

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature



def extract_start_points(typhoon_info_list):
    start_points = []
    for typhoon_info in typhoon_info_list:
        latitude_str = typhoon_info['latitude']
        longitude_str = typhoon_info['longitude']
        if latitude_str is not None and longitude_str is not None:
            latitude = float(latitude_str)
            longitude = float(longitude_str)
            start_point = [latitude, longitude]
            start_points.append(start_point)
    return np.array(start_points)
 
def visualize_start_points(start_points, region='globe2d'):
    # Set up the map projection and extent
    if region == 'globe2d':
        projection = ccrs.PlateCarree()
        extent = [-180, 180, -90, 90]
    elif region == 'southeastasia':
        projection = ccrs.PlateCarree()
        extent = [90, 180, 0, 45]
    elif region == 'globe3d':
        projection = ccrs.NearsidePerspective(central_longitude=180)
        extent = [-180, 180, -90, 90]

    fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': projection})
    ax.set_extent(extent)
    ax.add_feature(cfeature.COASTLINE, edgecolor='yellow')
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.gridlines(color='black', linestyle='dashed')

    ax.plot(start_points[:, 1], start_points[:, 0], marker='o', markersize=1, color='red', linestyle='', transform=ccrs.PlateCarree())

    plt.show()


import numpy as np

def extract_start_points_group(typhoon_info_list):
    start_points = []
    for typhoon_info in typhoon_info_list:
        start_lat = typhoon_info['latitude']
        start_lon = typhoon_info['longitude']
        grade = typhoon_info['grade']
        start_date = typhoon_info['start']
        year = int(start_date[:4])
        month = int(start_date[4:6])
        start_points.append([start_lat, start_lon, grade, year, month])
    return np.array(start_points)



def visualize_start_points_group_month(start_points):
    fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent([100, 180, 0, 50], crs=ccrs.PlateCarree())  # Adjust the extent for East Asia

    ax.add_feature(cfeature.COASTLINE, edgecolor='yellow')
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.gridlines(color='black', linestyle='dashed')

    unique_months = np.unique(start_points[:, 4])
    month_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
                    'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan',
                    'black']

    scatter_handles = []
    scatter_labels = []

    for i, month in enumerate(unique_months):
        month_color = month_colors[i % len(month_colors)]  # Cyclically repeat colors
        scatter = ax.scatter([], [], color=month_color)
        scatter_handles.append(scatter)
        scatter_labels.append(f"Month {month:2.0f}")

    for point in start_points:
        lat = point[0]
        lon = point[1]
        month = point[4]

        month_index = np.where(unique_months == month)[0][0]
        month_color = month_colors[month_index % len(month_colors)]  # Cyclically repeat colors

        ax.scatter(lon, lat, color=month_color, s=1, transform=ccrs.PlateCarree())

    ax.legend(scatter_handles, scatter_labels, loc='upper left')
    ax.set_title("Occurring Points - Lat/Lon")

    plt.show()


def visualize_start_points_group_grade(start_points):
    fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent([100, 180, 0, 50], crs=ccrs.PlateCarree())  # Adjust the extent for East Asia

    ax.add_feature(cfeature.COASTLINE, edgecolor='yellow')
    ax.add_feature(cfeature.LAND, facecolor='white')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.gridlines(color='black', linestyle='dashed')

    grades = start_points[:, 2]
    unique_grades = np.unique(grades[grades != None])
    grade_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
                    'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

    scatter_handles = []
    scatter_labels = []

    for i, grade in enumerate(unique_grades):
        grade_color = grade_colors[i % len(grade_colors)]  # Cyclically repeat colors
        scatter = ax.scatter([], [], color=grade_color)
        scatter_handles.append(scatter)
        scatter_labels.append(f"Grade {int(grade)}")

    for point in start_points:
        lat = point[0]
        lon = point[1]
        grade = point[2]

        if grade is not None:
            grade_index = np.where(unique_grades == grade)[0][0]
            grade_color = grade_colors[grade_index % len(grade_colors)]  # Cyclically repeat colors
            ax.scatter(lon, lat, color=grade_color, s=1, transform=ccrs.PlateCarree())

    ax.legend(scatter_handles, scatter_labels, loc='upper left')
    ax.set_title("Occurring Points - Lat/Lon")

    plt.show()


In [ ]:
# Assuming you have your typhoon_info_list
start_points = extract_start_points_group(typhoon_info_list)

In [ ]:
# Visualize the start points on a map
visualize_start_points_group_month(start_points)

In [ ]:
# Visualize the start points on a map
visualize_start_points_group_grade(start_points)